In [2]:
import os
from astropy.io import fits
from astropy.io import ascii as asc
from astropy.table import Table
import numpy as np

from matplotlib import pyplot as plt
%matplotlib
from utilities_az import spectroscopy as spec

Using matplotlib backend: Qt5Agg


### TODO: Should I be applying the dust correction, then deredshifting?

In [3]:
def read_iraf_spectrum(filename):
    ofile = fits.open(filename)
    flux = ofile[0].data[0,0,:]
    wave = spec.calc_wavelength(ofile[0].header, np.arange(len(flux))+1)
    rest_wave = spec.apply_redshift(wave, 0.0069)
    return(spec.spectrum1d(rest_wave, flux))

In [4]:
DATA_DIR_LCO = '../data/spectra/lco'
DATA_DIR_EFOSC = '../data/spectra/EFOSC'

In [5]:
spectra_files = [
         ('asassn15oz_20150904_redblu_122216.314.fits', DATA_DIR_LCO),
         ('asassn-15oz_20150906_redblu_105042.698a.fits', DATA_DIR_LCO),
         ('asassn15oz_20150907_redblu_123835.277.fits', DATA_DIR_LCO),
         ('asassn15oz_20150911_redblu_105336.349.fits', DATA_DIR_LCO),
         ('asassn15oz_20150916_redblu_120911.274.fits', DATA_DIR_LCO),
         ('asassn-15oz_20150920_redblu_135034.512.fits',DATA_DIR_LCO),
         ('asassn-15oz_20150924_redblu_123847.580.fits',DATA_DIR_LCO),
         ('asassn-15oz_20150930_redblu_122858.217.fits',DATA_DIR_LCO),
         ('tASASSN-15oz_20151003_Gr13_Free_slit1.0_57720_1_e.fits',DATA_DIR_EFOSC),
         ('asassn15oz_20151006_redblu_101906.800.fits', DATA_DIR_LCO),
         ('asassn15oz_20151014_redblu_112918.305.fits', DATA_DIR_LCO),
         ('asassn-15oz_20151025_redblu_102221.833.fits', DATA_DIR_LCO),
         ('asassn-15oz_20151107_redblu_101210.833.fits', DATA_DIR_LCO),
         ('tASAS-SN_15oz_20151107_Gr13_Free_slit1.5_57723_1_e.fits', DATA_DIR_EFOSC),
         ('tASAS-SN_15oz_20151118_Gr13_Free_slit1.0_57723_1_e.fits', DATA_DIR_EFOSC)
                ]

In [6]:
E_BV = 0.0804878 #From SNDAVIS

In [7]:
for ifile, idir in spectra_files:
    filename = os.path.join(idir, ifile)
    spectrum = read_iraf_spectrum(filename)
    dereddened_spec = spec.correct_for_galactic_extinction(spectrum, E_BV)
    tbdata = Table([dereddened_spec.wave, dereddened_spec.flux], names = ['rest_wavelength', 'dust_corr_flux'], meta={'Notes':'Rest wavelengths and dust corrected flux'})
    tbdata.write(filename.replace('.fits', '_dust_corr.asc'), format='ascii.commented_header', delimiter='\t', overwrite=True)

In [8]:
plt.plot(spectrum.wave, spectrum.flux, label='original')
plt.plot(dereddened_spec.wave, dereddened_spec.flux, label='dereddened')
plt.legend()

In [8]:
import extinction

In [9]:
extinction.apply?